# Implementing _discriminant analysis_ using Python and `numpy` <a class="tocSkip">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bayes'-theorem" data-toc-modified-id="Bayes'-theorem-1">Bayes' theorem</a></span></li><li><span><a href="#Gaussian-assumption" data-toc-modified-id="Gaussian-assumption-2">Gaussian assumption</a></span></li><li><span><a href="#LDA:-Assume-common-covariance" data-toc-modified-id="LDA:-Assume-common-covariance-3">LDA: Assume common covariance</a></span></li><li><span><a href="#QDA:-Class-specific-covariance" data-toc-modified-id="QDA:-Class-specific-covariance-4">QDA: Class-specific covariance</a></span></li><li><span><a href="#Fitting:-training-predictors-+-training-response---&gt;-model-paramenters" data-toc-modified-id="Fitting:-training-predictors-+-training-response--->-model-paramenters-5">Fitting: training predictors + training response --&gt; model paramenters</a></span></li><li><span><a href="#Predicting:-test-predictors----[model(parameters)]---&gt;-test-response" data-toc-modified-id="Predicting:-test-predictors----[model(parameters)]--->-test-response-6">Predicting: test predictors -- [model(parameters)] --&gt; test response</a></span></li><li><span><a href="#Test-implementation" data-toc-modified-id="Test-implementation-7">Test implementation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Fit" data-toc-modified-id="Fit-7.0.1">Fit</a></span></li><li><span><a href="#Fit-with-sklearn" data-toc-modified-id="Fit-with-sklearn-7.0.2">Fit with sklearn</a></span></li></ul></li><li><span><a href="#Test-against-sklearn:-fitting" data-toc-modified-id="Test-against-sklearn:-fitting-7.1">Test against sklearn: fitting</a></span></li><li><span><a href="#Test-against-sklearn:-predictions" data-toc-modified-id="Test-against-sklearn:-predictions-7.2">Test against sklearn: predictions</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np

# Bayes' theorem

By Bayes' theorem
$$
\begin{align}
P(y = k\ |\ x) & = \frac{P(y = k)\cdot P(x\ |\ y = k)}{P(x)}\\
& = \frac{\pi_k P(x\ |\ y = k)}{P(x)}\\
& = \frac{\pi_k P(x\ |\ y = k)}{\sum_{l=0}^K\pi_l P(x\ |\ y = k_l)}\tag{4.10}
\end{align}
$$

# Gaussian assumption
The multivariate Gaussian distribution is described by this function:
$$
f(x) = \frac{1}{(2\pi)^{p/2}|\mathbf{\Sigma}|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu)^T\mathbf{\Sigma}^{-1}(x-\mu)\right)\tag{4.18}
$$

Assume Gaussian distributions of predictors $X$ within each class $k$,
$$
P(x\ |\ y = k_l) = \frac{1}{(2\pi)^{p/2}|\mathbf{\Sigma}_l|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_l)^T\mathbf{\Sigma}_l^{-1}(x-\mu_l)\right)
$$

Putting into 4.10,
$$
\begin{align}
P(y = k\ |\ x) &= \frac{\pi_{k}\frac{1}{\cancel{(2\pi)^{p/2}}|\mathbf{\Sigma}_k|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_k)^T\mathbf{\Sigma}_k^{-1}(x-\mu_k)\right)}{\sum_{l=0}^K\pi_{l} \frac{1}{\cancel{(2\pi)^{p/2}}|\mathbf{\Sigma}_l|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_l)^T\mathbf{\Sigma}_l^{-1}(x-\mu_l)\right)}\\
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) &= \mathop{\mathrm{argmax}}_k \frac{\frac{\pi_{k}}{|\mathbf{\Sigma}_k|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_k)^T\mathbf{\Sigma}_k^{-1}(x-\mu_k)\right)}{\sum_{l=0}^K \frac{\pi_{l}}{|\mathbf{\Sigma}_l|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_l)^T\mathbf{\Sigma}_l^{-1}(x-\mu_l)\right)}\tag{i}\\
\end{align}
$$
where $\pi_k$ is the class prior, $\mu_k$ is the class mean, $\mathbf{\Sigma}_k$ is the class specific covariance matrix.

In [2]:
def class_prior(X, y):
    '''Return class priors.'''
    return np.array([len(X[y==k])/len(X) for k in np.unique(y)])

def class_mean(X, y):
    '''Return class means.'''
    return np.array([np.mean(X[y==k], axis=0) for k in np.unique(y)])

def class_cov(X, y):
    '''Return class covariances.'''
    return np.array([np.cov(X[y==k].T) for k in np.unique(y)])

# LDA: Assume common covariance
For LDA, we assume a common variance between all classes, i.e. $\mathbf{\Sigma}_0=\mathbf{\Sigma}_1=...\mathbf{\Sigma}_K$.

In [3]:
def weighted_cov(X, y):
    '''Return shared covariance, weighted by prior.'''
    
    cov = np.zeros((X.shape[1], X.shape[1]))
    priors = class_prior(X, y)
    
    for k, pi in zip(np.unique(y), priors):
        cov += np.cov(X[y==k].T, bias=True)*pi
        
    return cov

Thus, from (i),
$$
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) = \mathop{\mathrm{argmax}}_k \frac{\pi_{k}\exp\left(-\frac{1}{2}(x-\mu_k)^T(\mathbf{\Sigma})^{-1}(x-\mu_k)\right)}{\sum_{l=0}^K\pi_{l} \exp\left(-\frac{1}{2}(x-\mu_l)^T(\mathbf{\Sigma})^{-1}(x-\mu_l)\right)}\\
$$
$$
\begin{cases}
\pi \ge 0 \because \text{probability}\\
\exp(n) \gt 0 \because e \gt 0
\end{cases}
\implies \text{denominator} \ge 0
$$

And given $\text{denominator}\mathrel{\unicode{x2AEB}} k$,

$$
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) = \mathop{\mathrm{argmax}}_k \pi_{k}\exp\left(-(x-\mu_k)^T(2\mathbf{\Sigma})^{-1}(x-\mu_k)\right)
$$

$\because \log(x)$ is monotonically increasing,

$$
\begin{align}
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) &= \mathop{\mathrm{argmax}}_k \log P(y = k\ |\ x)\\
&= \mathop{\mathrm{argmax}}_k\log\left(\pi_{k}\exp\left(-\frac{1}{2}(x-\mu_k)^T(\mathbf{\Sigma})^{-1}(x-\mu_k)\right)\right)\\
&= \mathop{\mathrm{argmax}}_k\left(\log(\pi_k) -\frac{1}{2}(x-\mu_k)^T\mathbf{\Sigma}_k^{-1}(x-\mu_k)\right)\\
&= \mathop{\mathrm{argmax}}_k\left(\log(\pi_k) - \frac{1}{2} x^T \mathbf{\Sigma}_k^{-1} x + x^T \mathbf{\Sigma}_k^{-1} \mu_k - \frac{1}{2} \mu_k^T \mathbf{\Sigma}_k^{-1} \mu_k \right)\\
\end{align}
$$

And given $- \frac{1}{2} x^T \mathbf{\Sigma}_k^{-1} x \mathrel{\unicode{x2AEB}} k$,

$$
\begin{align}
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) &= \mathop{\mathrm{argmax}}_k \delta_k(x)\quad\text{where}\\
\delta_k(x) &= x^T \mathbf{\Sigma}_k^{-1} \mu_k - \frac{1}{2} \mu_k^T \mathbf{\Sigma}_k^{-1}\mu_k + \log(\pi_k) \tag{4.19, discriminant}\\
\end{align}
$$

# QDA: Class-specific covariance
For QDA, we do no assume common variance among classes. Thus, from (i), provided that

$$
\begin{cases}
\pi \ge 0 \because \text{probability}\\
|\mathbf{\Sigma}|^{-1/2} \ge 0 \because \text{square root}\\
\exp(n) \gt 0 \because e \gt 0
\end{cases}
\implies \text{denominator} \ge 0
$$

And given $\text{denominator}\mathrel{\unicode{x2AEB}} k$,

$$
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) = \mathop{\mathrm{argmax}}_k\left(\frac{\pi_k}{|\mathbf{\Sigma}_k|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_k)^T\mathbf{\Sigma}_k^{-1}(x-\mu_k)\right)\right)\\
$$

$\because \log(x)$ is monotonically increasing,

$$
\begin{align}
\mathop{\mathrm{argmax}}_k P(y = k\ |\ x) &= \mathop{\mathrm{argmax}}_k \log P(y = k\ |\ x)\\
&= \mathop{\mathrm{argmax}}_k\log\left(\frac{\pi_k}{|\mathbf{\Sigma}_k|^{1/2}}\exp\left(-\frac{1}{2}(x-\mu_k)^T\mathbf{\Sigma}_k^{-1}(x-\mu_k)\right)\right)\\
&= \mathop{\mathrm{argmax}}_k\left(\log(\pi_k) - \frac{1}{2}\log(|\mathbf{\Sigma}_k|) -\frac{1}{2}(x-\mu_k)^T\mathbf{\Sigma}_k^{-1}(x-\mu_k)\right)\\
&= \mathop{\mathrm{argmax}}_k\left(\log(\pi_k) - \frac{1}{2}\log(|\mathbf{\Sigma}_k|) - \frac{1}{2} x^T \mathbf{\Sigma}_k^{-1} x + x^T \mathbf{\Sigma}_k^{-1} \mu_k - \frac{1}{2} \mu_k^T \mathbf{\Sigma}_k^{-1} \mu_k \right)\\
&= \mathop{\mathrm{argmax}}_k \delta_k(x)\quad\text{where}\\
\delta_k(x) &= \color{red}{- \frac{1}{2} x^T \mathbf{\Sigma}_k^{-1} x} + x^T \mathbf{\Sigma}_k^{-1} \mu_k - \frac{1}{2} \mu_k^T \mathbf{\Sigma}_k^{-1}\mu_k \color{red}{- \frac{1}{2}\log(|\mathbf{\Sigma}_k|)} + \log(\pi_k) \tag{4.23, discriminant}\\
\end{align}
$$

In [4]:
def discriminant(X, prior, mean, cov, quadratic=False):
    '''Calculate discriminant delta(X)'''
    
    if len(X.shape) == 1:
        X = X.reshape(1,-1)
        
    d = np.dot(np.dot(X, np.linalg.pinv(cov)), mean) \
        - np.dot(np.dot(mean.T/2, np.linalg.pinv(cov)), mean) \
        + np.log(prior)
    
    # Extra terms for QDA (highlighted above in red)
    if quadratic:
        d -= np.array([np.dot(np.dot(X_i/2, np.linalg.pinv(cov)), X_i.T) \
                       for X_i in X])  # prevent matrix multiplication
        d -= np.log(np.linalg.det(cov))/2
        
    return d

def discriminants(X, priors, means, covs, quadratic=False):
    '''Calculate discriminant delta_k(X) per class k.'''

    dd = np.stack([discriminant(X, prior, mean, cov, quadratic) \
                   for prior, mean, cov in zip(priors, means, covs)], \
                  axis=-1)
    
    return dd

# Fitting: training predictors + training response --> model paramenters

In [5]:
def lda_fit(X, y):
    pi = class_prior(X,y)
    mu = class_mean(X,y)
    shared_sig = weighted_cov(X,y)
    return pi, mu, shared_sig

def qda_fit(X, y):
    pi = class_prior(X,y)
    mu = class_mean(X,y)
    sig = class_cov(X,y)
    return pi, mu, sig

# Predicting: test predictors -- [model(parameters)] --> test response

In [6]:
def lda_predict(x, pi, mu, shared_sig):
    sig = [shared_sig]*len(pi)  # Duplicate shared covariance.
    return np.argmax(discriminants(x, pi, mu, sig), axis=1)

def qda_predict(x, pi, mu, sig):
    return np.argmax(discriminants(x, pi, mu, sig, quadratic=True), axis=1)

# Test implementation

In [14]:
from sklearn import datasets

data = datasets.load_iris()
X = data.data[:,2:]
y = data.target
# y = (data.target > 0).astype(int)  # uncomment this for binary classification

print("n_observations (n): {}".format(X.shape[0]))
print("n_predictors (p): {}".format(X.shape[1]))
print("n_class (k): {}".format(len(np.unique(y))))

n_observations (n): 150
n_predictors (p): 2
n_class (k): 3


### Fit

In [15]:
priors_l, means_l, covar_l = lda_fit(X, y)
priors_q, means_q, covars_q = qda_fit(X, y)

### Fit with sklearn

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf_l = LinearDiscriminantAnalysis(store_covariance=True)
clf_q = QuadraticDiscriminantAnalysis(store_covariance=True)
clf_l.fit(X,y)
clf_q.fit(X,y)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=True, store_covariances=None, tol=0.0001)

## Test against sklearn: fitting

In [13]:
print('sklearn LDA parameters:\n', 'priors:',clf_l.priors_, 
      '\n means:', clf_l.means_, 
      '\n covariance:', clf_l.covariance_)

print('\n my LDA parameters:\n', 'priors:', priors_l, 
      '\n means:', means_l, 
      '\n covariance:', covar_l)

print('\n sklearn QDA parameters:\n', 'priors:',clf_q.priors_, 
      '\n means:', clf_q.means_, 
      '\n covariance:', np.array(clf_q.covariance_))

print('\n my QDA parameters:\n', 'priors:', priors_q, 
      '\n means:', means_q, 
      '\n covariance:', covars_q)

sklearn LDA parameters:
 priors: [0.33333333 0.33333333 0.33333333] 
 means: [[1.464 0.244]
 [4.26  1.326]
 [5.552 2.026]] 
 covariance: [[0.18146667 0.04169067]
 [0.04169067 0.04117067]]

 my LDA parameters:
 priors: [0.33333333 0.33333333 0.33333333] 
 means: [[1.464 0.244]
 [4.26  1.326]
 [5.552 2.026]] 
 covariance: [[0.18146667 0.04169067]
 [0.04169067 0.04117067]]

 sklearn QDA parameters:
 priors: [0.33333333 0.33333333 0.33333333] 
 means: [[1.464 0.244]
 [4.26  1.326]
 [5.552 2.026]] 
 covariance: [[[0.03010612 0.00569796]
  [0.00569796 0.01149388]]

 [[0.22081633 0.07310204]
  [0.07310204 0.03910612]]

 [[0.30458776 0.04882449]
  [0.04882449 0.07543265]]]

 my QDA parameters:
 priors: [0.33333333 0.33333333 0.33333333] 
 means: [[1.464 0.244]
 [4.26  1.326]
 [5.552 2.026]] 
 covariance: [[[0.03010612 0.00569796]
  [0.00569796 0.01149388]]

 [[0.22081633 0.07310204]
  [0.07310204 0.03910612]]

 [[0.30458776 0.04882449]
  [0.04882449 0.07543265]]]


## Test against sklearn: predictions 

In [11]:
my_predict_l = lda_predict(X, priors_l, means_l, covar_l)
sklearn_predict_l = clf_l.predict(X)

print('LDA predictions match:', (my_predict_l == sklearn_predict_l).all())

my_predict_q = qda_predict(X, priors_q, means_q, covars_q)
sklearn_predict_q = clf_q.predict(X)

print('QDA predictions match:', (my_predict_q == sklearn_predict_q).all())

LDA predictions match: True
QDA predictions match: True
